# Part 3: CQL syntax - exercises
### Goals
* To create more complex CQL queries
* To understand how to make some specific queries in Alma
    * Search by library
    * Search for inventory in institution
    * Special operator "=="
    * CQL limitations
   
### How to use SRU
#### SRU servers
* https://swisscovery.slsp.ch/view/sru/41SLSP_NETWORK?version=1.2&operation=explain
* https://renouvaud.primo.exlibrisgroup.com/view/sru/41BCULAUSA_NETWORK?version=1.2&operation=explain
* https://www.sudoc.abes.fr/cbs/sru/?operation=explain&version=1.1

#### Documentation
* https://slsp.ch/fr/metadonnees/
* https://developers.exlibrisgroup.com/alma/integrations/sru/
* https://knowledge.exlibrisgroup.com/Alma/Product_Documentation/010Alma_Online_Help_(English)/Metadata_Management/180Search_Indexes/051MARC_21_Bibliographic_All_Titles_Search
* https://knowledge.exlibrisgroup.com/Alma/Product_Documentation/Alma_Online_Help_(Francais)/130Int%C3%A9grations_avec_des_syst%C3%A8mes_externes/030Gestion_des_ressources/170Recherche_SRU%2F%2FSRW
* https://abes.fr/wp-content/uploads/2023/05/guide-utilisation-service-sru-catalogue-sudoc.pdf

In [3]:
import requests
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True, remove_comments=True, ns_clean=True)

ns = {
    "srw": "http://www.loc.gov/zing/srw/",
    "marc": "http://www.loc.gov/MARC21/slim"
}

In [4]:
# Example of code

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.holding_Library=112062990005504 and alma.title=Arbre',
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_UBS'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)

for record in xml.findall('.//marc:record', namespaces=ns):
    print('MMS ID: ', record.findtext('marc:controlfield[@tag="001"]', namespaces=ns))
    for ava_f in record.findall('.//marc:datafield[@tag="AVA"]', namespaces=ns):
        print('\t', ava_f.findtext('marc:subfield[@code="q"]', namespaces=ns), ' - ', ava_f.findtext('marc:subfield[@code="e"]', namespaces=ns))

MMS ID:  9961001430105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  9951928380105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  9941040010105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  9910722950105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  9967690830105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  993360990105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
	 Basel - Staatsarchiv Basel-Stadt  -  available
	 Basel - UB Hauptbibliothek  -  available
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
	 Basel - UB Hauptbibliothek  -  unavailable


### Some more advanced queries
* Find one library that has a volume of this journal (0022-2909)?
    1. Find one IZ, that has the title
    2. Find then one library in this IZ
* Find some analytical records of this journal (check 773w) - check Alma indexes

In [22]:
# Find one library that has a volume of this journal: 022-2909

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.issn=0022-2909',
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)

records = xml.findall('.//marc:record', namespaces=ns)
len(records)

2

In [23]:
for record in records:
    print([f022.text for f022 in record.findall('.//marc:datafield[@tag="022"]/marc:subfield[@code="a"]', namespaces=ns)])

['0022-2909', '1941-7497']
['1941-7497']


In [25]:
for f852 in xml.findall('.//marc:datafield[@tag="852"]', namespaces=ns):
    print(f852.findtext('marc:subfield[@code="a"]', namespaces=ns), f852.findtext('marc:subfield[@code="9"]', namespaces=ns))

41SLSP_UZB P
41SLSP_SBK P
41SLSP_BCUFR P
41SLSP_UBS P
41SLSP_HSG E
41SLSP_HPH E
41SLSP_RZH E
41SLSP_PHZ E
41SLSP_UZB E
41SLSP_ZHK E
41SLSP_RZS E
41SLSP_USI E
41SLSP_BFH E
41SLSP_IID E
41SLSP_UBS E
41SLSP_ZAW E
41SLSP_UGE E
41SLSP_FHO E
41SLSP_UNE E
41SLSP_FNW E
41SLSP_SUP E
41SLSP_ETH E
41SLSP_UBE E
41SLSP_BCUFR E
41SLSP_EPF E
41SLSP_VGE E
41SLSP_HES E


In [29]:
# We take for example 41SLSP_BCUFR

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.issn=0022-2909',
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_BCUFR'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)

records = xml.findall('.//marc:record', namespaces=ns)
len(records)

2

In [33]:
# Print version
for ava_f in xml.findall('.//marc:datafield[@tag="AVA"]/marc:subfield[@code="b"]', namespaces=ns):
    print(ava_f.text)

# Code of one library is "BCUF_BCU"

BCUF_BCU


In [38]:
# Online version
# m subfield contains the collection name
for ava_f in xml.findall('.//marc:datafield[@tag="AVE"]/marc:subfield[@code="m"]', namespaces=ns):
    print(ava_f.text)

JSTOR Archival Journals and Primary Sources Collection
Mirabel Libre accès


In [43]:
# Find some analytical records of this journal (check 773w for "(IDSBB)000010912DSV01") - check Alma indexes
# Check alma indexes: 773w is in "Other System Number"
# SRU index is "other_system_number"


# Find one library that has a volume of this journal: 022-2909 (system number: "(IDSBB)000010912DSV01")

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.other_system_number="(IDSBB)000010912DSV01"',
    'maximumRecords': '50'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)

records = xml.findall('.//marc:record', namespaces=ns)
len(records)

50

In [55]:
analytical_records = []
for record in records:
    if len(record.xpath('.//marc:datafield[@tag="773"]/marc:subfield[@code="w" and text()="(IDSBB)000010912DSV01"]', namespaces=ns)) > 0:
        analytical_records.append(record.findtext('marc:controlfield[@tag="001"]', namespaces=ns))

# Without paging, only 50 records can be found at once
analytical_records    

['991099791859705501',
 '991155872489705501',
 '991116048969705501',
 '991138286839705501',
 '991066726169705501',
 '991013868859705501',
 '991103594679705501',
 '991036991769705501',
 '991074173529705501',
 '991129765279705501',
 '991091929779705501',
 '991088399979705501',
 '991042095849705501',
 '991045633249705501',
 '991144551799705501',
 '991149650509705501',
 '991110311559705501',
 '991014299429705501',
 '991102976559705501',
 '991072813179705501',
 '991059834869705501',
 '991039414649705501',
 '991149495129705501',
 '991018050139705501',
 '991120655919705501',
 '991143100989705501',
 '991153027099705501',
 '991024723679705501',
 '991142980519705501',
 '991008807439705501',
 '991145078639705501',
 '991008670719705501',
 '991054025209705501',
 '991141558449705501',
 '991133367729705501',
 '991066729619705501',
 '991132972979705501',
 '991116714599705501',
 '991147289729705501',
 '991098659589705501',
 '991011155979705501',
 '991144357589705501',
 '991071188339705501',
 '991144936